*Copyright (c) Microsoft Corporation. All rights reserved.*

*Licensed under the MIT License.*

# Text Classification of MultiNLI Sentences using BERT

# Before You Start

The running time shown in this notebook is on a Standard_NC24s_v3 Azure Deep Learning Virtual Machine with 4 NVIDIA Tesla V100 GPUs. If you want to run through the notebook quickly, you can set the **`QUICK_RUN`** flag in the cell below to **`True`** to run the notebook on a small subset of the data and a smaller number of epochs. 
The table below provides some reference running time on different machine configurations.  

|QUICK_RUN|Machine Configurations|Running time|
|:---------|:----------------------|:------------|
|True|4 **CPU**s, 14GB memory| ~ 15 minutes|
|False|4 **CPU**s, 14GB memory| ~19.5 hours|
|True|1 NVIDIA Tesla K80 GPUs, 12GB GPU memory| |
|False|1 NVIDIA Tesla K80 GPUs, 12GB GPU memory| |

If you run into CUDA out-of-memory error, try reducing the `BATCH_SIZE` and `MAX_SEQ_LENGTH`, but note that model performance will be compromised. 

In [20]:
from datetime import datetime
startTime = datetime.now()

In [21]:
## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = False

TRAIN_DATA_USED_PERCENT = 1
TEST_DATA_USED_PERCENT = 1
NUM_EPOCHS = 1

In [22]:
if QUICK_RUN:
    TRAIN_DATA_USED_PERCENT = 0.01
    TEST_DATA_USED_PERCENT = 0.01
    NUM_EPOCHS = 1

import torch
if torch.cuda.is_available():
    BATCH_SIZE = 32
else:
    BATCH_SIZE = 8

In [23]:
import sys
sys.path.append("../../")
import os
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from utils_nlp.dataset.multinli import load_pandas_df
from utils_nlp.eval.classification import eval_classification
from utils_nlp.models.bert.sequence_classification import BERTSequenceClassifier
from utils_nlp.models.bert.common import Language, Tokenizer
from utils_nlp.common.timer import Timer
import torch
import torch.nn as nn
import numpy as np

## Introduction
In this notebook, we fine-tune and evaluate a pretrained [BERT](https://arxiv.org/abs/1810.04805) model on a subset of the [MultiNLI](https://www.nyu.edu/projects/bowman/multinli/) dataset.

We use a [sequence classifier](../../utils_nlp/models/bert/sequence_classification.py) that wraps [Hugging Face's PyTorch implementation](https://github.com/huggingface/pytorch-pretrained-BERT) of Google's [BERT](https://github.com/google-research/bert).

In [24]:
DATA_FOLDER = "../../../temp"
BERT_CACHE_DIR = "../../../temp"
LANGUAGE = Language.ENGLISH
TO_LOWER = True
MAX_LEN = 150
TRAIN_SIZE = 0.6
LABEL_COL = "genre"
TEXT_COL = "sentence1"

## Read Dataset
We start by loading a subset of the data. The following function also downloads and extracts the files, if they don't exist in the data folder.

The MultiNLI dataset is mainly used for natural language inference (NLI) tasks, where the inputs are sentence pairs and the labels are entailment indicators. The sentence pairs are also classified into *genres* that allow for more coverage and better evaluation of NLI models.

For our classification task, we use the first sentence only as the text input, and the corresponding genre as the label. We select the examples corresponding to one of the entailment labels (*neutral* in this case) to avoid duplicate rows, as the sentences are not unique, whereas the sentence pairs are.

In [25]:
df = load_pandas_df(DATA_FOLDER, "train")
df = df[df["gold_label"]=="neutral"]  # get unique sentences

In [26]:
df[[LABEL_COL, TEXT_COL]].head()

,genre,sentence1
0,government,Conceptually cream skimming has two basic dime...
4,telephone,yeah i tell you what though if you go price so...
6,travel,But a few Christian mosaics survive above the ...
12,slate,It's not that the questions they asked weren't...
13,travel,"Thebes held onto power until the 12th Dynasty,..."


The examples in the dataset are grouped into 5 genres:

In [27]:
df[LABEL_COL].value_counts()

telephone     27783
government    25784
travel        25783
fiction       25782
slate         25768
Name: genre, dtype: int64

We split the data for training and testing, and encode the class labels:

In [28]:
# split
df_train, df_test = train_test_split(df, train_size = TRAIN_SIZE, random_state=0)


In [29]:
df_train = df_train.sample(frac=TRAIN_DATA_USED_PERCENT).reset_index(drop=True)
df_test= df_test.sample(frac=TEST_DATA_USED_PERCENT).reset_index(drop=True)

In [30]:
# encode labels
label_encoder = LabelEncoder()
labels_train = label_encoder.fit_transform(df_train[LABEL_COL])
labels_test = label_encoder.transform(df_test[LABEL_COL])

num_labels = len(np.unique(labels_train))

In [31]:
print("Number of unique labels: {}".format(num_labels))
print("Number of training examples: {}".format(df_train.shape[0]))
print("Number of testing examples: {}".format(df_test.shape[0]))

Number of unique labels: 5
Number of training examples: 78540
Number of testing examples: 52360


## Tokenize and Preprocess

Before training, we tokenize the text documents and convert them to lists of tokens. The following steps instantiate a BERT tokenizer given the language, and tokenize the text of the training and testing sets.

In [32]:
tokenizer = Tokenizer(LANGUAGE, to_lower=TO_LOWER, cache_dir=BERT_CACHE_DIR)

tokens_train = tokenizer.tokenize(list(df_train[TEXT_COL]))
tokens_test = tokenizer.tokenize(list(df_test[TEXT_COL]))

100%|██████████| 52360/52360 [00:23<00:00, 2249.95it/s]


In addition, we perform the following preprocessing steps in the cell below:
- Convert the tokens into token indices corresponding to the BERT tokenizer's vocabulary
- Add the special tokens [CLS] and [SEP] to mark the beginning and end of a sentence
- Pad or truncate the token lists to the specified max length
- Return mask lists that indicate paddings' positions
- Return token type id lists that indicate which sentence the tokens belong to (not needed for one-sequence classification)

*See the original [implementation](https://github.com/google-research/bert/blob/master/run_classifier.py) for more information on BERT's input format.*

In [33]:
tokens_train, mask_train, _ = tokenizer.preprocess_classification_tokens(
    tokens_train, MAX_LEN
)
tokens_test, mask_test, _ = tokenizer.preprocess_classification_tokens(
    tokens_test, MAX_LEN
)

## Create Model
Next, we create a sequence classifier that loads a pre-trained BERT model, given the language and number of labels.

In [34]:
classifier = BERTSequenceClassifier(
    language=LANGUAGE, num_labels=num_labels, cache_dir=BERT_CACHE_DIR
)

## Train
We train the classifier using the training examples. This involves fine-tuning the BERT Transformer and learning a linear classification layer on top of that:

In [35]:
with Timer() as t:
    classifier.fit(
        token_ids=tokens_train,
        input_mask=mask_train,
        labels=labels_train,    
        num_epochs=NUM_EPOCHS,
        batch_size=BATCH_SIZE,    
        verbose=True,
    )    
print("[Training time: {:.3f} hrs]".format(t.interval / 3600))

t_total value of -1 results in schedule not being applied
Iteration:   0%|          | 1/9818 [00:06<18:45:44,  6.88s/it]

epoch:1/1; batch:1->982/9818; average training loss:1.770861


Iteration:  10%|█         | 983/9818 [1:41:41<16:17:55,  6.64s/it]

epoch:1/1; batch:983->1964/9818; average training loss:0.376318


Iteration:  20%|██        | 1965/9818 [3:24:56<13:57:37,  6.40s/it]

...

epoch:1/1; batch:7857->8838/9818; average training loss:0.246280


Iteration:  90%|█████████ | 8839/9818 [15:21:25<1:43:33,  6.35s/it]

epoch:1/1; batch:8839->9818/9818; average training loss:0.240945


Iteration: 100%|██████████| 9818/9818 [17:02:00<00:00,  5.65s/it]  

[Training time: 17.034 hrs]


## Score
We score the test set using the trained classifier:

In [36]:
preds = classifier.predict(
    token_ids=tokens_test, input_mask=mask_test, batch_size=BATCH_SIZE
)

Iteration: 100%|██████████| 6545/6545 [2:19:29<00:00,  1.24s/it]  


## Evaluate Results
Finally, we compute the accuracy, precision, recall, and F1 metrics of the evaluation on the test set.

In [37]:
print(classification_report(labels_test, preds, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

     fiction       0.94      0.88      0.91     10275
  government       0.89      0.96      0.93     10292
       slate       0.86      0.84      0.85     10277
   telephone       0.99      1.00      0.99     11205
      travel       0.96      0.96      0.96     10311

   micro avg       0.93      0.93      0.93     52360
   macro avg       0.93      0.93      0.93     52360
weighted avg       0.93      0.93      0.93     52360



In [38]:
print(datetime.now() - startTime)

19:22:59.651519
